In [ ]:
import databroker

In [ ]:
databroker.catalog()

In [ ]:
list(databroker.catalog())

In [ ]:
bc_catalog = databroker.catalog["bluesky-cartpole"]

In [ ]:
list(bc_catalog)

In [ ]:
training_run_ = bc_catalog["b21a"] # d78f, c406

In [ ]:
list(training_run_)

In [ ]:
training_run_data_ = training_run_.primary.read()

In [ ]:
training_run_data_

In [ ]:
training_run_data_.cartpole_average_evaluation_reward.plot()

In [ ]:
# how many episodes? count the values greater than 0 in cartpole terminal
episode_i_ = 0
episode_rewards_ = [0.0]
evaluation_episodes_ = []
evaluation_rewards_ = [0.0]
for reward_, terminal_, evaluation_reward_ in zip(
        training_run_data_.cartpole_reward,
        training_run_data_.cartpole_terminal,
        training_run_data_.cartpole_average_evaluation_reward):
    if terminal_ == 0:
        episode_rewards_[-1] += reward_
        episode_i_ += 1
    else:
        episode_rewards_.append(0.0)
    if evaluation_reward_ > 0.0:
        evaluation_rewards_.append(evaluation_reward_)
        evaluation_episodes_.append(episode_i_)
# the last episode is not real
episode_rewards_.pop(-1)
evaluation_rewards_.pop(-1)  # is this necessary? helpful?
print(f"episode rewards: {len(episode_rewards_)}")
print(f"evaluation rewards: {len(evaluation_rewards_)}")
print(f"evaluation episodes: {len(evaluation_episodes_)}")

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
episode_rewards_da_ = xr.DataArray(episode_rewards_)
episode_rewards_da_

In [ ]:
evaluation_rewards_da_ = xr.DataArray(evaluation_rewards_)
evaluation_rewards_da_

In [ ]:
figure, ax = plt.subplots()
ax.plot(range(episode_rewards_da_.shape[0]), episode_rewards_da_)
ax.plot(range(len(episode_rewards_)), episode_rewards_da_.rolling(dim_0=10).mean())

In [ ]:
tr_run_data = bc_catalog["b21a"].primary.read()

In [ ]:
tr_run_data.cartpole_average_evaluation_reward

In [ ]:
float(tr_run_data.cartpole_average_evaluation_reward[0])

In [ ]:
import datetime
def plot_episode_reward_for_training_run(partial_run_id, plot_point_count=-1):    
    training_run = bc_catalog[partial_run_id]

    print(training_run.metadata["start"])
    human_time = datetime.datetime.fromtimestamp(
        training_run.metadata["start"]["time"]
    ).isoformat()

    training_run_data = training_run.primary.read()
    training_run_data
    # how many episodes? count the values greater than 0 in cartpole terminal
    episode_i = 0
    evaluation_episodes = []
    episode_rewards = [0.0]
    evaluation_rewards = [0.0]
    for reward, terminal, evaluation_reward in zip(
        training_run_data.cartpole_reward,
        training_run_data.cartpole_terminal,
        training_run_data.cartpole_average_evaluation_reward
    ):
        if terminal == 0:
            episode_rewards[-1] += reward
        else:
            episode_rewards.append(0.0)
            episode_i += 1

        if evaluation_reward > 0.0:
            #print(f"evaluation reward: {evaluation_reward}")
            evaluation_rewards.append(float(evaluation_reward))
            evaluation_episodes.append(episode_i)
    # the last episode is not real
    episode_rewards.pop(-1)
    evaluation_rewards.pop(-1)
    print(f"{len(episode_rewards)} training episodes")
    print(f"{len(evaluation_rewards)} evaluations")

    episode_rewards_da = xr.DataArray(episode_rewards)
    episode_rewards_da_trimmed = episode_rewards_da[:plot_point_count]

    #print(f"evaluation episodes: {evaluation_episodes}")
    #print(f"evaluation rewards:  {evaluation_rewards}")
    
    #evaluation_rewards_da = xr.DataArray(evaluation_rewards)
    #evaluation_rewards_da_trimmed = evaluation_rewards_da[:plot_point_count]

    figure, ax = plt.subplots()
    ax.plot(
        range(episode_rewards_da_trimmed.shape[0]),
        episode_rewards_da_trimmed,
        label="training reward",
        alpha=0.5
    )
    ax.plot(
        range(episode_rewards_da_trimmed.shape[0]),
        episode_rewards_da_trimmed.rolling(dim_0=20).mean(),
        label="training reward moving average"
    )
    # how many evaluation episodes should be plotted?
    evaluation_episodes_trimmed = [i for i in evaluation_episodes if i < episode_rewards_da_trimmed.shape[0]]
    print(f"evaluation_episodes_trimmed: {evaluation_episodes_trimmed}")
    ax.plot(
        evaluation_episodes_trimmed,
        evaluation_rewards[:len(evaluation_episodes_trimmed)],
        label="evaluation reward"
    )

    #print("evaluation_rewards: ", evaluation_rewards)
    #print("evaluation_episodes: ", evaluation_episodes)

    agent = training_run.metadata["start"]["agent_name"].upper()
    ax.set_title(f"bluesky-cartpole {agent}")
    ax.set_xlabel("training episode")
    ax.set_ylabel("episode reward")
    ax.set_ylim(bottom=0, top=500)
    ax.legend()
    
    plt.savefig(f"bluesky-cartpole-{agent}-{episode_rewards_da_trimmed.shape[0]}.svg")

In [ ]:
### plot_episode_reward_for_training_run(partial_run_id="279a", plot_point_count=200)

In [ ]:
#plot_episode_reward_for_training_run(partial_run_id="1bcf", plot_point_count=200)

In [ ]:
plot_episode_reward_for_training_run(partial_run_id="b21a") #6eb?, c406

In [ ]:
plot_episode_reward_for_training_run(partial_run_id="ceea", plot_point_count=1000)

In [ ]:
plot_episode_reward_for_training_run(partial_run_id="c406", plot_point_count=200)

In [ ]:
plot_episode_reward_for_training_run(partial_run_id="ceea", plot_point_count=200)